This notebook generates a map to answer answer the following question:
* What facilities (by industry) are upstream of sample points with specific types of PFAS contamination within some range?

# Setup


In [17]:
%%capture
!pip install mapclassify
!pip install SPARQLWrapper
!pip install rdflib
!pip install ipython-autotime
%load_ext autotime

time: 22 s (started: 2025-11-19 01:43:20 +00:00)


In [18]:
from branca.element import Figure                                  # For controlling the size of the final map
import folium                                                      # For map layer control
import math                                                        # For math functions (specifically, math.log)
import geopandas as gpd                                            # For geospatial dataframes
import pandas as pd                                                # For dataframes
from shapely import wkt                                            # For working with WKT coordinates in a GeoDataFrame
from SPARQLWrapper import SPARQLWrapper2, JSON, GET, POST, DIGEST  # For querying SPARQL endpoints
import rdflib                                                      # For working with URIs

def convertToDataframe(results):
  d = []
  for x in results.bindings:
        row = {}
        for k in x:
            v = x[k]
            vv = rdflib.term.Literal(v.value, datatype=v.datatype).toPython()  # type: ignore[no-untyped-call]
            row[k] = vv
        d.append(row)
  df = pd.DataFrame(d)
  return df

def convertS2ListToQueryString(s2list):
  s2list_short = [s2cell.replace("http://stko-kwg.geog.ucsb.edu/lod/resource/","kwgr:") for s2cell in s2list]
  s2_values_string = " ".join(s2list_short)
  return s2_values_string


time: 2.15 ms (started: 2025-11-19 01:43:42 +00:00)


In [19]:
#for interactive widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

time: 1.54 ms (started: 2025-11-19 01:43:42 +00:00)


In [20]:
frink_fed_endpoint = 'https://frink.apps.renci.org/federation/sparql'

sparqlGET = SPARQLWrapper2(frink_fed_endpoint)
sparqlGET.setHTTPAuth(DIGEST)
sparqlGET.setMethod(POST)
sparqlGET.setReturnFormat(JSON)

time: 747 µs (started: 2025-11-19 01:43:42 +00:00)


# Q1 - What facilities are upstream from specific types of samples with high PFAS concentrations?


In [21]:
#Parameters to choose

substance = "PFOA" # @param ["PFOS", "PFOA", "PFBA", "PFBEA", "PFBS", "PFHPA", "PFHXS", "PFHXA", "PFHPS", "PFNA", "PFDA"]{"allow-input":true}
substanceCode = "me_egad:parameter." + substance + "_A"

materialType = "GW (Groundwater)" # @param ["DW (Drinking Water)", "GW (Groundwater)", "WW (Waste Water)", "SW (Surface Water)", "PW (Pore Water)", "L (Leachate)", "SR (Storm Water Runoff)", "SL (Soil)" ]{"allow-input":true}
matTypeCode = "me_egad_data:sampleMaterialType." + materialType.split()[0]

admin_region = "2301902795 (Bangor City, Maine)" # @param ["23 (Maine)","23019 (Penobscot County, Maine)","2301902795 (Bangor City, Maine)", "2301906925 (Brewer City, Maine)", "23011 (Kennebec County, Maine)", "23005 (Cumberland County, Maine)", "33 (New Hampshire)","17 (Illinois)"] {"allow-input":true}
regionCode = admin_region.split()[0]

minValue = 10 # @param
maxValue = 1000 # @param

#industry = "562 (Waste Treatment and Disposal)" # @param ["562 (Waste Treatment and Disposal)","322 (Paper Manufacturing)", "326 (Plastics and Rubber Products Manufacturing)","313 (Textile Mills)","321 (Wood Product Manufacturing)","325 (Chemical Manufacturing)", "336 (Aerospace Product and Part Manufacturing)"]{"allow-input":true}
#industryCode = industry.split()[0]



time: 1.86 ms (started: 2025-11-19 01:43:42 +00:00)


## Queries

In [22]:
# Query sample points, sample data, and S2 cells within a given administrative region

query = '''
PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
PREFIX fio: <http://w3id.org/fio/v1/fio#>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
PREFIX me_egad: <http://sawgraph.spatialai.org/v1/me-egad#>
PREFIX me_egad_data: <http://sawgraph.spatialai.org/v1/me-egad-data#>
PREFIX naics: <http://w3id.org/fio/v1/naics#>
PREFIX nhdplusv2: <http://nhdplusv2.spatialai.org/v1/nhdplusv2#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>

SELECT DISTINCT (COUNT(DISTINCT ?subVal) as ?resultCount) (MAX(?result_value) as ?max) ?sp ?spWKT ?upstream_flowlineWKT ?facility ?facWKT ?facilityName ?industryName ?industryGroup ?industryGroupName ?industrySubsector ?industrySubsectorName WHERE {
    ?sp rdf:type coso:SamplePoint ;
        geo:hasGeometry/geo:asWKT ?spWKT ;
        spatial:connectedTo ?ar3 ;
        spatial:connectedTo ?s2 . '''

if len(regionCode) > 5:
    query = query + ''' VALUES ?ar3 {<https://datacommons.org/browser/geoId/''' + regionCode +'''>}'''
else:
    query = query + ''' ?ar3 rdf:type kwg-ont:AdministrativeRegion_3 ;
         kwg-ont:administrativePartOf+ kwgr:administrativeRegion.USA.''' + regionCode + ''' . '''
query = query + '''
    ?s2 rdf:type kwg-ont:S2Cell_Level13 .
    ?s2cell rdf:type kwg-ont:S2Cell_Level13 ;
             kwg-ont:sfTouches | owl:sameAs ?s2 .
    ?observation rdf:type coso:ContaminantObservation ;
                coso:observedAtSamplePoint ?sp ;
                coso:ofSubstance ?substance ;
                coso:analyzedSample ?sample ;
                coso:hasResult ?result .
    ?sample rdfs:label ?sampleLabel ;
            coso:sampleOfMaterialType ?matType .
    ?matType rdfs:label ?matTypeLabel .
    ?result coso:measurementValue ?result_value ;
            coso:measurementUnit ?unit .
    ?unit qudt:symbol ?unit_sym .
    VALUES ?unit {<http://qudt.org/vocab/unit/NanoGM-PER-L>}
    VALUES ?substance {''' + substanceCode + '''}
    VALUES ?matType {''' + matTypeCode + '''}
    FILTER (?result_value > '''+ str(minValue) + ''')
    FILTER (?result_value <  '''+ str(maxValue) + ''')
    BIND((CONCAT(str(?result_value) , " ", ?unit_sym)) as ?subVal)

    ?downstream_flowline rdf:type hyf:HY_FlowPath ;
						 spatial:connectedTo ?s2cell .
	# find all flowlines upstream of them
	?upstream_flowline hyf:downstreamFlowPathTC ?downstream_flowline ;
					   geo:hasGeometry/geo:asWKT ?upstream_flowlineWKT .
	?s2cellus spatial:connectedTo ?upstream_flowline ;
			  rdf:type kwg-ont:S2Cell_Level13 .

    #find facilities
    ?s2cellus kwg-ont:sfContains ?facility.
    ?facility fio:ofIndustry ?industryCode, ?industryGroup, ?industrySubsector ;
              geo:hasGeometry/geo:asWKT ?facWKT;
              rdfs:label ?facilityName.
    ?industryCode a naics:NAICS-IndustryCode;
                  rdfs:label ?industryName ;
    fio:subcodeOf ?industryGroup .
    ?industryGroup a naics:NAICS-IndustryGroup;
                   rdfs:label ?industryGroupName ;
                   fio:subcodeOf ?industrySubsector .
    ?industrySubsector a naics:NAICS-IndustrySubsector;
                       rdfs:label ?industrySubsectorName;
                       fio:subcodeOf naics:NAICS-31 .

} GROUP BY ?sp ?spWKT ?upstream_flowlineWKT ?facility ?facWKT ?facilityName ?industryName ?industryGroup ?industryGroupName ?industrySubsector ?industrySubsectorName
'''

sparqlGET.setQuery(query)
results = sparqlGET.query()
results_df = convertToDataframe(results)
# print(results_df.columns)

time: 2.93 s (started: 2025-11-19 01:43:42 +00:00)


In [23]:
# # Use this query to pull all upstream flowlines
# # The first query only pulls the upstream flowlines near a facility
# # This query takes ~30 seconds

# query_hydro = '''
# PREFIX coso: <http://w3id.org/coso/v1/contaminoso#>
# PREFIX fio: <http://w3id.org/fio/v1/fio#>
# PREFIX geo: <http://www.opengis.net/ont/geosparql#>
# PREFIX hyf: <https://www.opengis.net/def/schema/hy_features/hyf/>
# PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
# PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
# PREFIX me_egad: <http://sawgraph.spatialai.org/v1/me-egad#>
# PREFIX me_egad_data: <http://sawgraph.spatialai.org/v1/me-egad-data#>
# PREFIX naics: <http://w3id.org/fio/v1/naics#>
# PREFIX nhdplusv2: <http://nhdplusv2.spatialai.org/v1/nhdplusv2#>
# PREFIX owl: <http://www.w3.org/2002/07/owl#>
# PREFIX qudt: <http://qudt.org/schema/qudt/>
# PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
# PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
# PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
# PREFIX spatial: <http://purl.org/spatialai/spatial/spatial-full#>

# SELECT ?upstream_flowline ?upstream_flowlineWKT WHERE {
#     ?sp rdf:type coso:SamplePoint ;
#         spatial:connectedTo ?ar3 ;
#         spatial:connectedTo ?s2 .
#     ?ar3 rdf:type kwg-ont:AdministrativeRegion_3 ;
#          kwg-ont:administrativePartOf kwgr:administrativeRegion.USA.''' + regionCode + ''' .
#     ?s2 rdf:type kwg-ont:S2Cell_Level13 .
#     ?s2cell rdf:type kwg-ont:S2Cell_Level13 ;
#              kwg-ont:sfTouches | owl:sameAs ?s2 .
#     ?observation rdf:type coso:ContaminantObservation ;
#                 coso:observedAtSamplePoint ?sp ;
#                 coso:ofSubstance ?substance ;
#                 coso:analyzedSample ?sample ;
#                 coso:hasResult ?result .
#     ?sample rdfs:label ?sampleLabel ;
#             coso:sampleOfMaterialType ?matType .
#     ?matType rdfs:label ?matTypeLabel .
#     ?result coso:measurementValue ?result_value ;
#             coso:measurementUnit ?unit .
#     ?unit qudt:symbol ?unit_sym .
#     VALUES ?unit {<http://qudt.org/vocab/unit/NanoGM-PER-L>}
#     VALUES ?substance {''' + substanceCode + '''}
#     VALUES ?matType {''' + matTypeCode + '''}
#     FILTER (?result_value > '''+ str(minValue) + ''')
#     FILTER (?result_value <  '''+ str(maxValue) + ''')
#     BIND((CONCAT(str(?result_value) , " ", ?unit_sym)) as ?subVal)

#     ?downstream_flowline rdf:type hyf:HY_FlowPath ;
# 						 spatial:connectedTo ?s2cell .
# 	?upstream_flowline hyf:downstreamFlowPathTC ?downstream_flowline ;
# 					   geo:hasGeometry/geo:asWKT ?upstream_flowlineWKT .
# }
# '''

# sparqlGET.setQuery(query_hydro)
# hydrology_results = sparqlGET.query()
# hydrology_df = convertToDataframe(hydrology_results)
# # print(results_df.columns)

time: 4 ms (started: 2025-11-19 01:43:45 +00:00)


In [24]:
# County boundaries for mapping
if len(regionCode) > 5: # select subdivision boundary
  county_query = '''
  PREFIX geo: <http://www.opengis.net/ont/geosparql#>
  PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

  SELECT * WHERE {
      ?county geo:hasGeometry/geo:asWKT ?countyWKT ;
              rdfs:label ?countyName.
      VALUES ?county {<https://datacommons.org/browser/geoId/'''+ regionCode + '''>}
  }
  '''
else:
  county_query = '''
  PREFIX geo: <http://www.opengis.net/ont/geosparql#>
  PREFIX kwgr: <http://stko-kwg.geog.ucsb.edu/lod/resource/>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

  SELECT * WHERE {
      ?county geo:hasGeometry/geo:asWKT ?countyWKT ;
              rdfs:label ?countyName.
      VALUES ?county {kwgr:administrativeRegion.USA.'''+ regionCode + '''}
  }
  '''

sparqlGET.setQuery(county_query)
county_results = sparqlGET.query()
county_df = convertToDataframe(county_results)

time: 339 ms (started: 2025-11-19 01:43:45 +00:00)


## Prep data for mapping

In [25]:
samplepoints_columns = [ 'sp', 'resultCount', 'max', 'spWKT' ]
samplepoints_df = results_df[samplepoints_columns].copy()
samplepoints_df.drop_duplicates(inplace=True)
samplepoints_df['spWKT'] = samplepoints_df['spWKT'].apply(wkt.loads)
samplepoints_df = gpd.GeoDataFrame(samplepoints_df, geometry='spWKT')
samplepoints_df.set_crs(epsg=4326, inplace=True, allow_override=True)

,sp,resultCount,max,spWKT
0,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,62.6,POINT (-68.83307 44.80625)
2,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,96.2,POINT (-68.82163 44.81341)
4,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,290.0,POINT (-68.82705 44.81171)
6,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,343.0,POINT (-68.82652 44.81115)
8,http://sawgraph.spatialai.org/v1/me-egad-data#...,1,490.0,POINT (-68.82563 44.81127)
10,http://sawgraph.spatialai.org/v1/me-egad-data#...,2,34.6,POINT (-68.83394 44.80626)
12,http://sawgraph.spatialai.org/v1/me-egad-data#...,2,123.0,POINT (-68.82221 44.81629)


time: 40.6 ms (started: 2025-11-19 01:43:45 +00:00)


In [26]:
# # Use this block instead of the following one if only interested in some (not all) upstream flowlines

hydrology_columns = [ 'upstream_flowlineWKT' ]
hydrology_df = results_df[hydrology_columns].copy()
hydrology_df.drop_duplicates(inplace=True)
hydrology_df['upstream_flowlineWKT'] = hydrology_df['upstream_flowlineWKT'].apply(wkt.loads)
hydrology_df = gpd.GeoDataFrame(hydrology_df, geometry='upstream_flowlineWKT')
hydrology_df.set_crs(epsg=4326, inplace=True, allow_override=True)

,upstream_flowlineWKT
0,"LINESTRING (-68.84163 44.80553, -68.84185 44.8..."
12,"LINESTRING (-68.78954 44.88405, -68.78957 44.8..."
14,"LINESTRING (-68.82903 44.85901, -68.82893 44.8..."
16,"LINESTRING (-68.82903 44.8551, -68.82864 44.85..."
18,"LINESTRING (-68.8313 44.86283, -68.83131 44.86..."
20,"LINESTRING (-68.84093 44.84058, -68.8399 44.84..."
22,"LINESTRING (-68.84291 44.87237, -68.84283 44.8..."
24,"LINESTRING (-68.93948 44.93679, -68.93867 44.9..."
28,"LINESTRING (-68.95516 44.93861, -68.95287 44.9..."
32,"LINESTRING (-69.01704 45.00267, -69.01713 45.0..."


time: 33.6 ms (started: 2025-11-19 01:43:45 +00:00)


In [27]:
# # Use this block instead of the preceding one if attempting to map all upstream flowlines

# hydrology_df.drop_duplicates(inplace=True)
# hydrology_df['upstream_flowlineWKT'] = hydrology_df['upstream_flowlineWKT'].apply(wkt.loads)
# hydrology_df = gpd.GeoDataFrame(hydrology_df, geometry='upstream_flowlineWKT')
# hydrology_df.set_crs(epsg=4326, inplace=True, allow_override=True)

time: 608 µs (started: 2025-11-19 01:43:45 +00:00)


In [28]:
facilities_columns = [ 'facility', 'facWKT', 'facilityName', 'industryName', 'industryGroup', 'industryGroupName', 'industrySubsector', 'industrySubsectorName' ]
facilities_df = results_df[facilities_columns].copy()
facilities_df.drop_duplicates(inplace=True)
facilities_df['facWKT'] = facilities_df['facWKT'].apply(wkt.loads)
facilities_df = gpd.GeoDataFrame(facilities_df, geometry='facWKT')
facilities_df.set_crs(epsg=4326, inplace=True, allow_override=True)

,facility,facWKT,facilityName,industryName,industryGroup,industryGroupName,industrySubsector,industrySubsectorName
0,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.8492 44.80031),DOWNEAST CONCRETE PRODUCTS INC,Concrete Block and Brick Manufacturing,http://w3id.org/fio/v1/naics#NAICS-3273,Cement and Concrete Product Manufacturing,http://w3id.org/fio/v1/naics#NAICS-327,Nonmetallic Mineral Product Manufacturing
1,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.8492 44.80031),DOWNEAST CONCRETE PRODUCTS INC,"Concrete Pipe, Brick, and Block Manufacturing",http://w3id.org/fio/v1/naics#NAICS-3273,Cement and Concrete Product Manufacturing,http://w3id.org/fio/v1/naics#NAICS-327,Nonmetallic Mineral Product Manufacturing
12,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.82706 44.85969),"FORCE MANUFACTURING, INC.","Industrial Truck, Tractor, Trailer, and Stacke...",http://w3id.org/fio/v1/naics#NAICS-3339,Other General Purpose Machinery Manufacturing,http://w3id.org/fio/v1/naics#NAICS-333,Machinery Manufacturing
13,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.82706 44.85969),"FORCE MANUFACTURING, INC.",Material Handling Equipment Manufacturing,http://w3id.org/fio/v1/naics#NAICS-3339,Other General Purpose Machinery Manufacturing,http://w3id.org/fio/v1/naics#NAICS-333,Machinery Manufacturing
16,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.76987 44.80374),CONSOLIDATED COMMUNICATIONS O,"Capacitor, Resistor, Coil, Transformer, and Ot...",http://w3id.org/fio/v1/naics#NAICS-3344,Semiconductor and Other Electronic Component M...,http://w3id.org/fio/v1/naics#NAICS-334,Computer and Electronic Product Manufacturing
17,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.76987 44.80374),CONSOLIDATED COMMUNICATIONS O,Semiconductor and Other Electronic Component M...,http://w3id.org/fio/v1/naics#NAICS-3344,Semiconductor and Other Electronic Component M...,http://w3id.org/fio/v1/naics#NAICS-334,Computer and Electronic Product Manufacturing
24,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.93252 44.92765),"NORTHEASTERN LOG HOMES, LLC",All Other Wood Product Manufacturing,http://w3id.org/fio/v1/naics#NAICS-3219,Other Wood Product Manufacturing,http://w3id.org/fio/v1/naics#NAICS-321,Wood Product Manufacturing
25,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.93252 44.92765),"NORTHEASTERN LOG HOMES, LLC",Prefabricated Wood Building Manufacturing,http://w3id.org/fio/v1/naics#NAICS-3219,Other Wood Product Manufacturing,http://w3id.org/fio/v1/naics#NAICS-321,Wood Product Manufacturing
26,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.93252 44.92765),"NORTHEASTERN LOG HOMES, LLC",Sawmills,http://w3id.org/fio/v1/naics#NAICS-3211,Sawmills and Wood Preservation,http://w3id.org/fio/v1/naics#NAICS-321,Wood Product Manufacturing
27,http://w3id.org/fio/v1/epa-frs-data#d.FRS-Faci...,POINT (-68.93252 44.92765),"NORTHEASTERN LOG HOMES, LLC",Sawmills and Wood Preservation,http://w3id.org/fio/v1/naics#NAICS-3211,Sawmills and Wood Preservation,http://w3id.org/fio/v1/naics#NAICS-321,Wood Product Manufacturing


time: 59.1 ms (started: 2025-11-19 01:43:45 +00:00)


In [29]:
county_df['countyWKT'] = county_df['countyWKT'].apply(wkt.loads)
county_df = gpd.GeoDataFrame(county_df, geometry='countyWKT')
county_df.set_crs(epsg=4326, inplace=True, allow_override=True)

,county,countyWKT,countyName
0,https://datacommons.org/browser/geoId/2301902795,"POLYGON ((-68.85904 44.86494, -68.85697 44.865...","Bangor city, Penobscot County, Maine"


time: 29.2 ms (started: 2025-11-19 01:43:45 +00:00)


In [30]:
%%capture
map = county_df.explore(name='Counties',
                        style_kwds=dict(color='Gray',
                                        fill=0.0,
                                        weight=7))

samplepoints_df.explore(m=map,
                        name=f'<span style="color:DarkOrange;">Samples</span>',
                        color='DarkOrange',
                        style_kwds=dict(style_function=lambda x: { 'radius': math.log(float(x['properties']['max'])) * 1.8976 + 3.36937,  # fits to (4 ppt, 6 radius) and (6400 ppt, 20 radius)
                                                                   'opacity': 0.3,
                                                                   'color': 'DimGray',
                                                                 }
                                       ),
                        marker_kwds=dict(radius=6),
                        marker_type='circle_marker',
                        popup = [ 'sp', 'resultCount', 'max' ],
                        show=False)

hydrology_df.explore(m=map,
                     name='<span style="color:Blue;">Flowlines</span>',
                     color='Blue',
                     style_kwds=dict(weight=.5),
                     show=False)

c = 0
colors = ['MidnightBlue','MediumBlue','SlateBlue','MediumSlateBlue', 'DodgerBlue','DeepSkyBlue','SkyBlue','CadetBlue','DarkCyan','LightSeaGreen',
          'MediumSageGreen','PaleVioletRed','Purple','Orchid','Fuchsia','MediumVioletRed','HotPink','LightPink', 'lightblue', 'gray', 'blue', 'darkred',
          'lightgreen', 'purple', 'red', 'green', 'lightred', 'white', 'darkblue', 'darkpurple', 'cadetblue', 'orange', 'pink', 'lightgray', 'darkgreen']
for industry in list(facilities_df.industrySubsectorName.unique()):
    facilities_df[facilities_df['industrySubsectorName']==industry].explore(m=map,
                                                                            name=f'<span style="color:{colors[c]};">{industry}</span>',
                                                                            color=colors[c],
                                                                            marker_kwds=dict(radius=6,
                                                                                             fill=True),
                                                                            style_kwds=dict(fillOpacity=1),
                                                                            marker_type='circle_marker',
                                                                            popup=True,
                                                                            show=False)
    c += 1

time: 178 ms (started: 2025-11-19 01:43:45 +00:00)


## Map

In [31]:
folium.LayerControl(collapsed=False).add_to(map)
fig = Figure(width='100%', height=700)
fig.add_child(map)

time: 110 ms (started: 2025-11-19 01:43:45 +00:00)


In [32]:
fig.save('SAWGraph-demo_2025-09-28_TracingUpstream_Kennebec.html')

time: 95.3 ms (started: 2025-11-19 01:43:45 +00:00)
